<a href="https://colab.research.google.com/github/JulianaCarvajal/Spaceship_Titanic/blob/workOnPreprocess%2FBustamJos3/dataset_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#libraries
from zipfile import ZipFile
import pandas as pd
import numpy as np
#call API
%pip install kaggle
#convert categorical to numerical
from sklearn.preprocessing import LabelEncoder
#oneHotEncoding
from sklearn.preprocessing import OneHotEncoder
# nan imputation
from sklearn.impute import KNNImputer
# standardization is required 'cause many models work badly is data does'nt look like normally distributed data (see biblio)
from sklearn.preprocessing import StandardScaler

# Working with the next cell [Online]
For now on, import cell from colab will have this code (**On local notebook, respective changes must be made in order to import .json token and datasets from kaggle**).

In [ ]:
#upload kaggle token .json
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bustamjos3","key":"a80d020fd1917115be9ce5ce749b71d9"}'}

In [ ]:
#Make a directory named kaggle and copy the kaggle.json file there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# change the permission of the file
!chmod 600 ~/.kaggle/kaggle.json
# Import data files from the Kaggle API
!kaggle competitions download -c spaceship-titanic
#unzip files
file_name = "spaceship-titanic.zip" #the file is the dataset exact name
with ZipFile("spaceship-titanic.zip", 'r') as zip:
  zip.extractall()
  print('Done')
# .zip is now not necessary
!rm *.zip

spaceship-titanic.zip: Skipping, found more recently modified local copy (use --force to force download)
Done


# End of working [Online]

# Working on Local

In [ ]:
#call API
%pip install kaggle
# Import data files from the Kaggle API
!kaggle competitions download -c spaceship-titanic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Unzip the data files
with ZipFile("spaceship-titanic.zip", 'r') as zip_ref:
    zip_ref.extractall()
    print('Done')
# .zip is now not necessary
!rm *.zip

# End of working on Local

In [ ]:
# Read the data files into pandas dataframes
d_sTitanic = pd.read_csv("train.csv")
d_sTitanic.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


# Pipeline for work on preprocessing

In [ ]:
#convert types of cols to the best possible dtype
d_sTitanic=d_sTitanic.convert_dtypes(infer_objects=True)

In [ ]:
d_sTitanic.dtypes

PassengerId      string
HomePlanet       string
CryoSleep       boolean
Cabin            string
Destination      string
Age               Int64
VIP             boolean
RoomService       Int64
FoodCourt         Int64
ShoppingMall      Int64
Spa               Int64
VRDeck            Int64
Name             string
Transported     boolean
dtype: object

## Drop at 1st looking, not relevant columns:
* ```d_sTitanic['PassengerId']```
* ```d_sTitanic['Name']```

In [ ]:
#drop those columns
d_sTitanic.drop(['PassengerId','Name'],axis=1,inplace=True)

In [ ]:
# separation of X, y
y=d_sTitanic.iloc[:,-1].values

In [ ]:
#also drop
d_sTitanic.drop(['Transported'],axis=1,inplace=True)

In [ ]:
# take name cols which are categorical
listCategoric=[str(i) for i in (d_sTitanic.dtypes=='string').loc[(d_sTitanic.dtypes=='string')==True].index]
# take the categoric types only
#to handle oneHotEncoder, replace NaN values with 'wanted'
XCategoric=d_sTitanic[listCategoric].fillna('NaN').values
#numeric col names
listNumeric=[str(i) for i in (d_sTitanic.dtypes=='Int64').loc[(d_sTitanic.dtypes=='Int64')==True].index]
# take the numeric types only
XNumeric=d_sTitanic[listNumeric].fillna(-1).values

### Transformation on ```Cabin``` with labelEncoder

In [ ]:
#instancing
labelEncoder=LabelEncoder()
XCategoric[:,1]=labelEncoder.fit_transform(XCategoric[:,1]) #labelEncoding

## oneHotEncoding


In [ ]:
# instancing
oneHotEncoder=OneHotEncoder(handle_unknown='error',sparse=False)
#fitting with categorical X
oneHotEncoder.fit(XCategoric[:,[0,2]]) #array-like object with categories of the n-categoric cols
convertedOHE=oneHotEncoder.transform(XCategoric[:,[0,2]]) # tranformation to get convertion to OHE

### Concatenate oneHotEncoded cols with ```Cabin``` col

In [ ]:
# .reshape on XCategoric for dimensionality problems
convertedOHE= np.concatenate( (convertedOHE[:,:len(oneHotEncoder.categories_[0])-1], XCategoric[:,1].reshape((-1,1)), convertedOHE[:,len(oneHotEncoder.categories_[0])-1:7 ]), axis=1 )

## ```NaN``` Imputation

In [ ]:
#instancing
kNNImputer=KNNImputer(n_neighbors=1, missing_values=-1,weights='distance')
#imputation
imputedNumeric=kNNImputer.fit_transform(XNumeric)

### Now merge ```imputedNumeric``` and ```convertedOHE```

In [ ]:
# using numpy.concantenate
X=np.concatenate( (imputedNumeric, convertedOHE), axis=1 )

array([[39.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [24.0, 109.0, 9.0, ..., 0.0, 0.0, 0.0],
       [58.0, 43.0, 3576.0, ..., 0.0, 0.0, 0.0],
       ...,
       [26.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [32.0, 0.0, 1049.0, ..., 1.0, 0.0, 0.0],
       [44.0, 126.0, 4688.0, ..., 0.0, 0.0, 0.0]], dtype=object)

In [ ]:
# scaling with train
standardS=StandardScaler().fit(X)

## Categorical values converted to OHE shoulb be standardized?

In [ ]:
XScaled=standardS.transform(X)
XScaled

array([[ 0.69747552, -0.33679313, -0.28266174, ..., -0.51101319,
        -0.146233  , -0.31748665],
       [-0.34078331, -0.17326511, -0.27702934, ..., -0.51101319,
        -0.146233  , -0.31748665],
       [ 2.01260337, -0.27228208,  1.95527526, ..., -0.51101319,
        -0.146233  , -0.31748665],
       ...,
       [-0.2023488 , -0.33679313, -0.28266174, ..., -0.51101319,
        -0.146233  , -0.31748665],
       [ 0.21295473, -0.33679313,  0.37382482, ...,  1.95689664,
        -0.146233  , -0.31748665],
       [ 1.0435618 , -0.14776074,  2.65118856, ..., -0.51101319,
        -0.146233  , -0.31748665]])

# End of work on preprocessing

In [ ]:
# export preprocessed X to .json
pprocessedX=pd.DataFrame( XScaled )
pprocessedX.to_json('pprocessedX.json', orient='index')